In [91]:
import pandas as pd
import json
import requests
import datetime
import calendar

In [92]:
coin = requests.get('https://api.coincap.io/v2/assets?limit=30') #limit 30 because coins are ranked already
coin = json.loads(coin.text)
coins_df = pd.json_normalize(coin['data'])

In [93]:
coins_df.head()

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer
0,bitcoin,1,BTC,Bitcoin,18720075.0000000000000000,21000000.0000000000000000,703983634048.0590125180240550,21161394698.9909645443649339,37605.8126929544359474,0.1611428393802610,38937.9855957004252893,https://blockchain.info/
1,ethereum,2,ETH,Ethereum,116052830.9365000000000000,None,310647572825.6073105543938815,19085930579.1264883870871919,2676.7772084386520764,0.0625246883520432,2767.7114456867633287,https://etherscan.io/
2,tether,3,USDT,Tether,61125907361.9716900000000000,None,61210747790.9835624952484011,57643131488.1030429639856367,1.0013879618753055,-0.0845930071220385,1.0012201375607313,https://www.omniexplorer.info/asset/31
3,binance-coin,4,BNB,Binance Coin,153432897.0000000000000000,170532785.0000000000000000,54806115500.0439920017701897,1930963058.2596534231657880,357.1992484769677001,2.3055912564060647,369.9188492622870784,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,cardano,5,ADA,Cardano,31948309440.7478000000000000,45000000000.0000000000000000,52214422411.4589782761127197,3179396954.9637378623765882,1.6343406998826420,-1.7878059642577821,1.6912814496039256,https://cardanoexplorer.com/


In [94]:
currentdate = datetime.datetime.utcnow()
current_year = currentdate.year
endDate = datetime.datetime.timestamp(currentdate)*1000
start_datetime = datetime.datetime(current_year, 1, 1, 0, 0, 0, 1)
startDate = datetime.datetime.timestamp(start_datetime)*1000

In [101]:
assets = list()

for coinId in coins_df.id:
    url = "https://api.coincap.io/v2/assets/" + coinId + "/history?interval=d1&start=" + str(startDate) + "&end=" + str(endDate)
    endpoint = requests.get(url)
    data = json.loads(endpoint.text)
    result = {'id': coinId, 
              'history': data['data']}
    assets.append(result)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df_assets = pd.DataFrame(assets)

In [ ]:
ratesPerDay = list()

for asset in assets:
    coinHistory = asset['history']
    coinId = asset['id']

    for day in coinHistory:  
        a = {'date': day['date'], 'coin':coinId, 'rateUsd': day['priceUsd']}
        ratesPerDay.append(a)

In [ ]:
df_rates = pd.DataFrame(ratesPerDay)

In [ ]:
euro = requests.get('https://api.coincap.io/v2/rates/euro')
euro = json.loads(euro.text)
df_euro = pd.DataFrame([euro['data']])
df_euro

In [ ]:
euro_list = list()
for row in df_rates:
    df_rates['rate_usd_to_eur'] = df_euro['rateUsd']

In [ ]:
df_rates.to_csv("ninetynine_rates.csv")